In [ ]:
"""
Remember to change up the initialization
"""

In [1]:
#Imports:

import dcgan
import lie

import torch
import torch.utils.data as data
import torch.optim as optim
from torch.autograd import Variable

import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torchvision.datasets.vision import VisionDataset

import os
import lmdb
from PIL import Image
import io
import math

In [19]:
def print_settings(settings):
    #logs a settings (dict) to a file, which is specified in the settings
    #assumes the root folder already exists?
    if "root" in settings.keys():
        f = open(settings["root"]+"/settings.txt","w")
        for k in settings.keys():
            f.write(k + " : " + str(settings[k]) + "\n")
        f.close()
    else:
        print("Can't find where to place the settings.")

settings = {
    "imageSize"      : 64,
    "batchSize"      : 64,
    "workers"        : 4,
    "ngpu"           : 1,           #Number of gpus to use
    "nz"             : 100,         #size of latent z vector,
    "ngf"            : 64,
    "ndf"            : 64,
    "nc"             : 3,           #input image channels,
    "n_extra_layers" : 0,           #number of extra layers on gen and disc,
    "root"           : "generated_samples/S1_test_init", #Name of folder to save results to I think?,
    "niter"          : 25,          #Number of epochs maybe?,
    "use_LIE"        : True,        #use the LIE net?,
    "clip"           : 0.01, #default 0.01,
    "GlearningRate"   : 0.00005,
    "DlearningRate"   : 0.009, #default 0.00005,
    "gen_iterations"  : math.inf,
}

#Might as well make the folder now
if not os.path.isdir(settings["root"]):
    os.mkdir(settings["root"])
print_settings(settings)

In [3]:
class dataset_partial(dset.VisionDataset):
    """
    A class that only loads data when we need it; so it doesn't
    have ridiculous memory requirements. 
    """
    def __init__(self,root, transform=None, target_transform=None):
        
        super(dataset_partial, self).__init__(root, transform=transform,
                                   target_transform=target_transform)
        self.classes = self._verify_classes([])
        
        self.env = lmdb.open(root, max_readers=1, readonly=True, lock=False,
                             readahead=False, meminit=False)
        with self.env.begin() as txn:
            self.length = int(txn.stat()['entries'])
    
    def _verify_classes(self,classes):
        return ['bedroom_train']
    
    def __getitem__(self,index):
        
        with self.env.begin(write=False) as txn:
            cursor = txn.cursor()
            if not cursor.set_range(str(index).encode()): # Position at first key >= '5'.
                print('Not found!')
            else:
                for key,_ in cursor:
                    imgbuf = txn.get(key)
                    break
        
        buf = io.BytesIO()
        buf.write(imgbuf)
        buf.seek(0)
        img = Image.open(buf).convert('RGB')
        img = self.transform(img)
        
        return img,0

    def __len__(self):
        return self.length

In [4]:
#Initializing the data loaders:

lsun_dataset = dataset_partial("/data1/users/jzlin/ML_DATASETS/lsun/bedroom_train_lmdb",
                               transform=transforms.Compose([
                                transforms.Scale(settings["imageSize"]),
                                transforms.CenterCrop(settings["imageSize"]),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                            ]))

dataloader = torch.utils.data.DataLoader(lsun_dataset, 
                                         num_workers = settings["workers"],
                                         batch_size  = settings["batchSize"])

/home/jzlin/.local/lib/python3.5/site-packages/torchvision/transforms/transforms.py:209: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


In [5]:
"""
A picture of the discriminator network

  (*,3,64,64) -> (*,64,32,32) -> (*,128,16,16) -> (*,256,8,8) -> (*,512,4,4) -> (*,1,1,1)
          (64,3,4,4)     (128,64,4,4)    (256,128,4,4)   (512,256,4,4)   (1,512,4,4)

"""

'\nA picture of the discriminator network\n\n  (*,3,64,64) -> (*,64,32,32) -> (*,128,16,16) -> (*,256,8,8) -> (*,512,4,4) -> (*,1,1,1)\n          (64,3,4,4)     (128,64,4,4)    (256,128,4,4)   (512,256,4,4)   (1,512,4,4)\n \n\n\n'

In [20]:
settings["root"]

'generated_samples/S1_test_init'

In [ ]:
train_1()

/home/jzlin/.local/lib/python3.5/site-packages/ipykernel_launcher.py:99: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


500


/home/jzlin/.local/lib/python3.5/site-packages/ipykernel_launcher.py:136: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
32000
32500
33000
33500
34000
34500
35000
35500
36000
36500
37000
37500
38000
38500
39000
39500
40000
40500
41000
41500
42000
42500
43000
43500
44000
44500
45000
45500
46000
46500
47000
47500
48000
48500
49000
49500
50000
50500
51000
51500
52000
52500
53000
53500
54000
54500
55000
55500
56000
56500
57000
57500
58000
58500
59000
59500
60000
60500
61000
61500
62000
62500
63000
63500
64000
64500
65000
65500
66000
66500
67000
67500
68000
68500
69000
69500
70000
70500
71000
71500
72000
72500
73000
73500
74000
74500
75000
75500
76000
76500
77000
77500
78000
78500
79000
79500
80000
80500
81000
81500
82000
82500
83000
83500
84000
84500
85000
8550

In [7]:
for lr in [0.02,0.009,0.03,0.008]:
    settings["DlearningRate"] = lr
    settings["root"] = "generated_samples/S1_scan_bias_clipped/" + str(lr)
    
    #Might as well make the folder now
    if not os.path.isdir(settings["root"]):
        os.mkdir(settings["root"])
    print_settings(settings)
    
    train_1()

NameError: name 'train_1' is not defined

In [16]:

def train_1():
    #Defining the Network here. 

    # custom weights initialization called on netG and netD
    def weights_init(m):
        classname = m.__class__.__name__
        if classname.find('Conv') != -1:
            m.weight.data.normal_(0.0, 0.02)
        elif classname.find('BatchNorm') != -1:
            m.weight.data.normal_(1.0, 0.02)
            m.bias.data.fill_(0)

    netG = dcgan.DCGAN_G(settings["imageSize"],
                         settings["nz"], 
                         settings["nc"], 
                         settings["ngf"], 
                         settings["ngpu"], 
                         settings["n_extra_layers"])
    netG.apply(weights_init)

    if settings["use_LIE"] == True:
        netD = lie.DCGAN_D_LIE(settings["imageSize"],
                               settings["nz"],
                               settings["nc"],
                               settings["ndf"],
                               settings["ngpu"],
                               settings["n_extra_layers"],
                               clip=settings["clip"])#,clip=clip)
    else:
        netD = dcgan.DCGAN_D(settings["imageSize"],
                             settings["nz"], 
                             settings["nc"], 
                             settings["ndf"], 
                             settings["ngpu"], 
                             settings["n_extra_layers"])
    netD.apply(weights_init)

    input = torch.FloatTensor(settings["batchSize"], 3, settings["imageSize"], settings["imageSize"]) 
    noise = torch.FloatTensor(settings["batchSize"], settings["nz"], 1, 1)
    fixed_noise = torch.FloatTensor(settings["batchSize"], settings["nz"], 1, 1).normal_(0, 1) #first arg batchsize
    one = torch.FloatTensor([1])
    mone = one * -1

    netD.cuda()
    netG.cuda()
    input = input.cuda()
    one, mone = one.cuda(), mone.cuda()
    noise, fixed_noise = noise.cuda(), fixed_noise.cuda()

    optimizerD = optim.RMSprop(netD.parameters(), lr = settings["DlearningRate"])
    optimizerG = optim.RMSprop(netG.parameters(), lr = settings["GlearningRate"])

    gen_iterations = 0

    #actually training. 

    for epoch in range(25):
        data_iter = iter(dataloader)
        i = 0
        while i < len(dataloader):
            ############################
            # (1) Update D network
            ###########################
            for p in netD.parameters(): # reset requires_grad
                p.requires_grad = True # they are set to False below in netG update

            # train the discriminator Diters times
            if gen_iterations < 25 or gen_iterations % 500 == 0:
                Diters = 100
            else:
                Diters = 5
            j = 0
            while j < Diters and i < len(dataloader):
                j += 1

                # clamp parameters to a cube
                if settings["use_LIE"] == False:
                    for p in netD.parameters():
                        p.data.clamp_(-settings["clip"], settings["clip"])

                data = data_iter.next()
                i += 1

                # train with real
                real_cpu, _ = data
                netD.zero_grad()
                batch_size = real_cpu.size(0)

                #if opt.cuda:
                real_cpu = real_cpu.cuda()
                input.resize_as_(real_cpu).copy_(real_cpu)
                inputv = Variable(input)

                errD_real = netD(inputv)
                errD_real.backward(one)

                # train with fake
                noise.resize_(settings["batchSize"], settings["nz"], 1, 1).normal_(0, 1) #first arg batch size
                noisev = Variable(noise, volatile = True) # totally freeze netG
                fake = Variable(netG(noisev).data)
                inputv = fake
                errD_fake = netD(inputv)
                errD_fake.backward(mone)
                errD = errD_real - errD_fake
                optimizerD.step()

            ############################
            # (2) Update G network
            ###########################
            for p in netD.parameters():
                p.requires_grad = False # to avoid computation
            netG.zero_grad()
            # in case our last batch was the tail batch of the dataloader,
            # make sure we feed a full batch of noise
            noise.resize_(settings["batchSize"], settings["nz"], 1, 1).normal_(0, 1) #first arg batch size
            noisev = Variable(noise)
            fake = netG(noisev)
            errG = netD(fake)
            errG.backward(one)
            optimizerG.step()
            gen_iterations += 1

            #print('[%d/%d][%d/%d][%d] Loss_D: %f Loss_G: %f Loss_D_real: %f Loss_D_fake %f'
            #    % (epoch, niter, i, len(dataloader), gen_iterations,
            #    errD.data[0], errG.data[0], errD_real.data[0], errD_fake.data[0]))

            #if gen_iterations %20000 == 0:
            #    print('[%d/%d][%d/%d][%d] Loss_D: %f Loss_G: %f Loss_D_real: %f Loss_D_fake %f'
            #    % (epoch, niter, i, len(dataloader), gen_iterations,
            #    errD.data[0], errG.data[0], errD_real.data[0], errD_fake.data[0]))
            #print(gen_iterations)
            if gen_iterations % 500 == 0:
                print(gen_iterations)
                real_cpu = real_cpu.mul(0.5).add(0.5)
                vutils.save_image(real_cpu, '{0}/real_samples.png'.format(settings["root"]))
                fake = netG(Variable(fixed_noise, volatile=True))
                fake.data = fake.data.mul(0.5).add(0.5)
                vutils.save_image(fake.data, '{0}/fake_samples_{1}.png'.format(settings["root"], gen_iterations))
            
            if gen_iterations >= settings["gen_iterations"]:
                break
        if gen_iterations >= settings["gen_iterations"]:
            break

        # do checkpointing
        torch.save(netG.state_dict(), '{0}/netG_epoch_{1}.pth'.format(settings["root"], epoch))
        torch.save(netD.state_dict(), '{0}/netD_epoch_{1}.pth'.format(settings["root"], epoch))